In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.chat_models import ChatOpenAI

In [6]:
chat = ChatOpenAI(openai_api_base=os.environ['OPENAI_REVERSE_PROXY'],
                  openai_api_key=os.environ["OPENAI_API_KEY"],
                  temperature=0,
                  model = 'gpt-3.5-turbo')

In [9]:
# Create chat messages to supply to the chat model

from langchain.schema import (SystemMessage,HumanMessage,AIMessage)

messages = [
    SystemMessage(content="You ar ea helpful assistant"),
    HumanMessage(content="Hello, how is the weather?"),
    AIMessage(content="It currenly 32 degress celcius and its raining"),
    HumanMessage(content="That is some groggy weather!!"),
    AIMessage(content="Yes, it can be quite gloomy and uncomfortable when it's hot and rainy. Is there anything else I can assist you with?"),
    HumanMessage(content="Talking of weather, I would like to know more about the weather prediction models, I mean how they predict the weather quite accurately?")

]

In [10]:
#pass it to the llm

res = chat(messages=messages)
res.content

"Weather prediction models are complex computer algorithms that use mathematical equations to simulate and forecast the behavior of the atmosphere. These models take into account various factors such as temperature, humidity, wind patterns, and atmospheric pressure to predict future weather conditions.\n\nThe models are based on the laws of physics and use a technique called numerical weather prediction (NWP). NWP involves dividing the atmosphere into a three-dimensional grid and solving mathematical equations that describe the physical processes occurring within each grid cell. These equations represent the conservation of mass, momentum, and energy, and they are solved iteratively to simulate the evolution of the atmosphere over time.\n\nTo initialize the models, meteorologists collect vast amounts of observational data from weather stations, satellites, radars, and other sources. This data is used to create an initial snapshot of the atmosphere's current state, known as the initial 

In [11]:
# Add the last ai response to the messages

messages.append(res)

In [13]:
#Now create a new user prompt and then append that to the messages and pass it to the chat

prompt = HumanMessage(content="Well assume we have to predict the weather on sea. How do we get the initial data from?")
messages.append(prompt)


#send it to chat-gpt
res = chat(messages=messages)

## New Prompt Template

In [23]:
# Setup the system message

messages = [
    SystemMessage(content=(
        'You are a helpful assistant. You keep responses to no more than '
        '100 words long (including whitespace), and sign off every '
        'message with a random name like "Robot McRobot" or "Bot Rob".'
    )),
    HumanMessage(content="Hi AI, please explan what are the weather models and how these models are used to predict the weather")
]

In [26]:
res = chat(messages)
print(f"length: {len(res.content)}\n{res.content}")


#Clearly we can see that the openai chat is not at all following the system messages, hence we wil try to use the different promopt templates 

from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate

human_template = HumanMessagePromptTemplate.from_template(
    '{input} Can you keep the response to no more than 100 words'+
    '(including whitespace), and sign off with a random name like "Robot ' + 'McRobot" or "Bot Rob".'
)


#create the human message

chat_prompt = ChatPromptTemplate.from_messages([human_template])

#format with some input

chat_prompt_value = chat_prompt.format_prompt(input="I would like to know more about the weather prediction models, I mean how they predict the weather quite accurately?")

length: 621
Weather models are complex computer simulations that use mathematical equations to predict how atmospheric conditions will evolve over time. They factor in data like temperature, pressure, humidity, and wind patterns to simulate the atmosphere's behavior. These models help meteorologists forecast weather by projecting future conditions based on current observations. By analyzing various scenarios, models can estimate how weather systems, like storms or fronts, will develop. This aids in generating forecasts for different timeframes and locations, enhancing our understanding of upcoming weather patterns.

- Bot Rob


In [28]:
chat_prompt_value.to_messages()

[HumanMessage(content='I would like to know more about the weather prediction models, I mean how they predict the weather quite accurately? Can you keep the response to no more than 100 words(including whitespace), and sign off with a random name like "Robot McRobot" or "Bot Rob".', additional_kwargs={}, example=False)]

In [29]:
chat_prompt_value.to_string()

'Human: I would like to know more about the weather prediction models, I mean how they predict the weather quite accurately? Can you keep the response to no more than 100 words(including whitespace), and sign off with a random name like "Robot McRobot" or "Bot Rob".'

In [34]:
messages = [
    SystemMessage(content=(
        'You are a helpful assistant. You keep responses to no more than '
        '100 characters long (including whitespace), and sign off every '
        'message with a random name like "Robot McRobot" or "Bot Rob".'
    )),
    chat_prompt.format_prompt(
        input="Hi AI, What are weather models, and how weather is predicted?"
    ).to_messages()[0]
]


res =  chat(messages)


In [35]:
print(f"length: {len(res.content)}\n{res.content}")

length: 288
Weather models are computer simulations that use mathematical equations to predict future weather conditions. They analyze current data like temperature, humidity, and wind to make forecasts. These models help meteorologists understand atmospheric processes and make predictions. Bot Rob.


In [41]:
#We can also use the prompt templates approach for building and initial system message with a few examples for the chatbot to follow

from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate
)


system_template = SystemMessagePromptTemplate.from_template(
    'You are a helpful assistant. You keep responses to no more than '
    '{character_limit} characters long (including whitespace), and sign '
    'off every message with "- {sign_off}'
)
human_template = HumanMessagePromptTemplate.from_template("{input}")
ai_template = AIMessagePromptTemplate.from_template("{response} - {sign_off}")

# create the list of messages
chat_prompt = ChatPromptTemplate.from_messages([
    system_template,
    human_template,
    ai_template
])
# format with required inputs
chat_prompt_value = chat_prompt.format_prompt(
    character_limit="50", sign_off="Robot McRobot",
    input="Hi AI, how are you? What is quantum physics?",
    response="Good! It's physics of small things"
)
chat_prompt_value



ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant. You keep responses to no more than 50 characters long (including whitespace), and sign off every message with "- Robot McRobot', additional_kwargs={}), HumanMessage(content='Hi AI, how are you? What are the weather models?', additional_kwargs={}, example=False), AIMessage(content="Good! It's physics of small things - Robot McRobot", additional_kwargs={}, example=False)])

In [49]:
messages = chat_prompt_value.to_messages()
messages.append(HumanMessage(content="What are weather models and how are they used to predict the weather"))

res = chat(messages)
print(f"length: {len(res.content)}\n{res.content}")
#Hence the key is to deliver some examples

length: 82
Weather models are simulations. They use math and data to forecast - Robot McRobot


In [50]:
from langchain import PromptTemplate

human_template = HumanMessagePromptTemplate.from_template('{input} Answer in less than {character_limit} characters (including whitespace).')

#create the human message

human_prompt = ChatPromptTemplate.from_messages([human_template])

#Format with some input

human_prompt_value = human_prompt.format_prompt(input="Okay cool, so who built this models?",character_limit = '50')

human_prompt_value

ChatPromptValue(messages=[HumanMessage(content='Okay cool, so who built this models? Answer in less than 50 characters (including whitespace).', additional_kwargs={}, example=False)])

In [51]:
messages.pop(-1)

HumanMessage(content='What are weather models and how are they used to predict the weather', additional_kwargs={}, example=False)

In [53]:
messages.extend(human_prompt_value.to_messages())
messages

[SystemMessage(content='You are a helpful assistant. You keep responses to no more than 50 characters long (including whitespace), and sign off every message with "- Robot McRobot', additional_kwargs={}),
 HumanMessage(content='Hi AI, how are you? What are the weather models?', additional_kwargs={}, example=False),
 AIMessage(content="Good! It's physics of small things - Robot McRobot", additional_kwargs={}, example=False),
 HumanMessage(content='How small?', additional_kwargs={}, example=False),
 HumanMessage(content='What are weather models and how are they used to predict the weather', additional_kwargs={}, example=False),
 HumanMessage(content='Okay cool, so who built this models? Answer in less than 50 characters (including whitespace).', additional_kwargs={}, example=False),
 HumanMessage(content='Okay cool, so who built this models? Answer in less than 50 characters (including whitespace).', additional_kwargs={}, example=False)]

In [54]:
res = chat(messages)
print(f"length {len(res.content)}\n{res.content}")

length 45
Scientists and meteorologists - Robot McRobot


In [55]:
# Now try this with simple f-strings

_input = "Okay cool , so who created these weather models"
character_limit = 50

human_message = HumanMessage(content=(
    f"{_input} Answer in less than {character_limit} characters "
    "(including whitespace)."
))

human_message

HumanMessage(content='Okay cool , so who created these weather models Answer in less than 50 characters (including whitespace).', additional_kwargs={}, example=False)

In [56]:
messages.pop(-1)

messages.append(human_message)

In [57]:
messages

[SystemMessage(content='You are a helpful assistant. You keep responses to no more than 50 characters long (including whitespace), and sign off every message with "- Robot McRobot', additional_kwargs={}),
 HumanMessage(content='Hi AI, how are you? What are the weather models?', additional_kwargs={}, example=False),
 AIMessage(content="Good! It's physics of small things - Robot McRobot", additional_kwargs={}, example=False),
 HumanMessage(content='How small?', additional_kwargs={}, example=False),
 HumanMessage(content='What are weather models and how are they used to predict the weather', additional_kwargs={}, example=False),
 HumanMessage(content='Okay cool, so who built this models? Answer in less than 50 characters (including whitespace).', additional_kwargs={}, example=False),
 HumanMessage(content='Okay cool , so who created these weather models Answer in less than 50 characters (including whitespace).', additional_kwargs={}, example=False)]

In [58]:
res = chat(messages)
print(f"Length: {len(res.content)}\n{res.content}")

Length: 45
Scientists and meteorologists - Robot McRobot
